### Este Script corre solo sin tener que traer ningun dataset, generando observaciones de clase 0 y 1 con distribucion uniforme [0,1] para cada uno de sus puntos. 

Para correrlo con datos reales, se pueden pisar la matriz X y el vector y directamente (celda 3)



In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [42]:
observaciones = 5000
puntos_por_observacion = 20
random_state=1 # mantener este valor para reproductibilidad de los resultados

In [43]:
X = np.random.rand(observaciones, puntos_por_observacion)
y = np.random.randint(0, 2, size=observaciones)
print(X.shape)
print(y.shape)

(5000, 20)
(5000,)


In [44]:
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, test_size=0.2, random_state=random_state)
print('Train X: '+str(X_train.shape))
print('Train y: '+str(y_train.shape))
print('Dev X: '+str(X_dev.shape))
print('Dev y: '+str(y_dev.shape))
print('Test X: '+str(X_test.shape))
print('Test y: '+str(y_test.shape))

Train X: (3200, 20)
Train y: (3200,)
Dev X: (800, 20)
Dev y: (800,)
Test X: (1000, 20)
Test y: (1000,)


Vamos a usar DTW (Dynamic Time Warping) para comparar las series temporales
Que es esto?: https://towardsdatascience.com/dynamic-time-warping-3933f25fcdd

In [45]:
###  OJO!! Cada instancia de test tarda unos 10 segundos en predecir. 

cuantas_instancias_de_test_corremos = 10


cuantas_instancias_de_test_corremos = min(cuantas_instancias_de_test_corremos, len(X_test))
predicciones = np.zeros(cuantas_instancias_de_test_corremos)

for i in range(len(predicciones)):

  distancia_acumualda = np.zeros(2)

  for j in range(len(X_train)):
    distancia_acumualda[y_train[j]] += fastdtw(X_train[j], X_test[i], dist=euclidean)[0]

  distancia_acumualda[0] /= len(y_train[y_train==0])
  distancia_acumualda[1] /= len(y_train[y_train==1])

  if distancia_acumualda[1] < distancia_acumualda[0]:
    predicciones[i] = 1


Comparemos el ground truth con lo predicho. En el caso de muestras generadas al azar, deberia dar cerca de 0.5 (si corremos unos cuantos ejemplos de test)

In [46]:
predicciones_correctas = np.sum(y_test[:cuantas_instancias_de_test_corremos] == predicciones)

print("Predicciones correctas: {} %".format(str(round(100 * predicciones_correctas / cuantas_instancias_de_test_corremos, 3))))

Predicciones correctas: 70.0 %
